In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [2]:
df1 = pd.read_csv('../input/the-movies-dataset/ratings.csv', low_memory=False)
#df2 = pd.read_csv('../input/the-movies-dataset/links_small.csv', low_memory=False)
df3 = pd.read_csv('../input/the-movies-dataset/credits.csv', low_memory=False)
df4 = pd.read_csv('../input/the-movies-dataset/keywords.csv', low_memory=False)
df5 = pd.read_csv('../input/the-movies-dataset/movies_metadata.csv', low_memory=False)
#df6 = pd.read_csv('../input/the-movies-dataset/ratings_small.csv', low_memory=False)
#df7 = pd.read_csv('../input/the-movies-dataset/links.csv', low_memory=False)

* Dataset 1 - ratings(26024289)
    * userId
    * movieId
    * rating
    * timestamp
* Dataset 2 - links_small(9125)
    * movieId
    * imdbId
    * tmdbId
* Dataset 3 - credits(45476)\
    * cast
    * crew
    * id - movieId

* Dataset 4 - keywords(46419)
    * id - movieId
    * keywords
* Dataset 5 - movies_metadata(45466)
    * adult - is movie adult rated ?
    * belongs_to_collection - 
    * budget
    * genres
    * homepage - homepage link
    * id - movieId
    * imdb_id
    * original_language
    * original_title
    * overview
    * popularity
    * poster_path
    * production_companies
    * production_countries
    * release_date
    * revenue
    * runtime
    * spoken_languages
    * status
    * tagline
    * title
    * video
    * vote_average - rating out of 10
    * vote_count - total number of votes
* Dataset 6 - ratings_small(100004)
    * userId
    * movieId
    * rating
    * timestamp
* Dataset 7 - links(45843)
    * movieId
    * imdbId
    * tmdbId

In [3]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

**Updates-**
* drop values without a title
* popularity, budget, id - convert to numeric

In [4]:
df5.dropna(subset=['title'], inplace=True)
df5['popularity'] = pd.to_numeric(df5['popularity'])
df5['budget'] = pd.to_numeric(df5['budget'])
df5['id'] = pd.to_numeric(df5['id'])

# update genres column from json to list
df5['genres'] = df5['genres'].apply(lambda x: ' '.join([i['name'] for i in eval(x)]))

In [5]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [6]:
# updates cast and crew columns from json to list
df3['cast'] = df3['cast'].apply(lambda x: ' '.join([i['name'].replace(' ', '') for i in eval(x)]))
df3['crew'] = df3['crew'].apply(lambda x: ' '.join([i['name'].replace(' ', '') for i in eval(x)]))

In [7]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [8]:
# update keywords from json to list
df4['keywords'] = df4['keywords'].apply(lambda x: ' '.join([i['name'] for i in eval(x)]))

# Recommendor 1 : Demographic Filtering
Generalized recommendation to all users

## 1. Based on Popularity (Trending movies)

In [9]:
data = df5.sort_values('popularity', ascending=False)
fig = px.bar(data.head(10), x='popularity', y='title', orientation='h', hover_name='overview')
fig.update_yaxes(autorange='reversed')
fig.show()

## 2. Based on average rating & number of votes (IMDB Rating)
generating a new metric score as per the imdb weighted rating<br>
**weighted rating (WR)** = (v / (v+m)) × R + (m / (v+m)) × C<br>
where:<br> 
R = average for the movie (mean) = (Rating) <br>
v = number of votes for the movie = (votes) <br>
m = minimum votes required to be listed in the Top 250 (more votes than 99% of movies)<br>
C = mean vote across all movies

In [10]:
data = df5.copy()
m = data['vote_count'].quantile(0.99)
data = data[data['vote_count']>m]
print('m =', m)
print(data.shape)
R = data['vote_average']
v = data['vote_count']
C = data['vote_average'].mean()
data['weighted rating'] = (v/(v+m))*R + (m/(v+m))*C

m = 2183.8199999999924
(455, 24)


In [11]:
data = data.sort_values('weighted rating', ascending=False)
fig = px.bar(data.head(10), x='weighted rating', y='title', orientation='h', hover_data=['vote_count', 'vote_average'])
fig.update_yaxes(autorange='reversed')
fig.show()

# Recommender 2 : Content Based Filtering
Recommending based on the content of the movie (overview, cast, crew, keyword, tagline, genre etc)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet as wn
import re
from functools import lru_cache
import string
from tqdm import tqdm
tqdm.pandas()

In [13]:
wnl     = WordNetLemmatizer()
porter  = PorterStemmer()

def tree_2_word(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN

# memonization - to increase speed n efficiency
lemmatize_mem = lru_cache(maxsize=16384)(wnl.lemmatize)

def lemmatize(word, pos=wn.NOUN):
    return(lemmatize_mem(word, pos))

def text_process(t, pos=True, lemmetize=True, stem=True):
    # Defining stop words
    stops = set(stopwords.words("english"))
    # Remove linebreak, tab, return  
    t = re.sub('[\n\t\r]+', ' ', t)      
    # Remove Punctuations  
    t = re.sub('['+string.punctuation+']+', '', t)
    t = re.sub('\s+\s+', ' ', t)                             # Remove double whitespace
    t= re.sub(r'\\', ' ', t)                                # Remove \ slash
    t= re.sub(r'\/', ' ', t)                                # Remove / slash
    
    # Convert to lower case
    t = t.lower()             
    # Remove Non-letters                            
    t = re.sub('[^a-zA-Z]',' ',t)  
    # Sentence wise tokenize: required otherwise pos tagging will not work properly                       
    sentence = nltk.sent_tokenize(t)                   
    modified_sentence = ""
    for s in sentence:
        # Word Tokenization 
        words = nltk.word_tokenize(s)     
        # checking if pos is required                
        if pos:               
            # Part of speech Tagging                            
            tag_words = pos_tag(words)                        
            lemmatized = " "
            for tw in tag_words:
                # checking stop word
                if tw[0] not in stops:
                    # lemmatization
                    if lemmetize:                         
                        lemma = lemmatize(tw[0], tree_2_word(tw[1]))  
                        # check if lemmetizer word is present in the wordnet library if not present then stemming is required
                        # example running not in synsets: stem to run  
                        # stemming check if word is in library then perform otherwise skip
                        if (stem) & (wn.synsets(lemma)==[]) & (wn.synsets(porter.stem(lemma))!=[]):
                            # Stemming
                            lemma = porter.stem(tw[0])    
                    else:
                        if stem:
                            lemma = porter.stem(tw[0])
                            if not wn.synsets(lemma):
                                lemma = tw[0]
                        else:
                            lemma = tw[0]
                    lemmatized = lemmatized+" "+lemma
        else:
            lemmatized = " "
            for w in words:
                if w not in stops:
                    if lemmetize:                         # lemmatization
                        lemma = lemmatize(w)
                        if (stem) & (wn.synsets(lemma)==[]) & (wn.synsets(porter.stem(lemma))!=[]):
                            # Stemming
                            lemma = porter.stem(w)    
                    else:
                        if stem:
                            lemma = porter.stem(w)
                            if not wn.synsets(lemma):
                                lemma = w
                        else:
                            lemma = w
                    lemmatized = lemmatized+" "+lemma               
        modified_sentence = modified_sentence+" "+lemmatized
    return modified_sentence

## 1. Based on Overview

In [14]:
# get top 15000 movies based on popularity
data = df5.sort_values('popularity', ascending=False).reset_index(drop=True)
data = data.iloc[:15000].copy()

# drop duplicates based on title
data.drop_duplicates(subset=['title'], inplace=True)

# replace NaN values with empty string
data['overview'] = data['overview'].fillna('')

# cleaning the text
data['overview'] = data['overview'].progress_apply(text_process)

100%|██████████| 14383/14383 [01:38<00:00, 145.78it/s]


In [15]:
# convert overview column data to matrix to be able to compute similarity after removing the stop words
tfidf = TfidfVectorizer(ngram_range=(1,2),min_df=0.0001)
tfidf_matrix = tfidf.fit_transform(data['overview'])

print(tfidf_matrix.shape)

(14383, 51197)


In [16]:
# compute similarity matrix
# options available - euclidean distance, the Pearson Coeffiecient, cosine similarity score

# compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# create reverse title indices series
indices = pd.Series(data.index, index=data['title'])

In [17]:
def get_recommendation(title):
    # get index of given title
    idx = indices[title]
    
    # get similarity scores along with indices
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # sort the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # get top 10 similarity scores
    sim_scores = sim_scores[1:11]
    
    # get movie indices
    movie_indices = [i[0] for i in sim_scores]
    movie_scores = [i[1] for i in sim_scores]
    
    # get movie titles
    movies = data.iloc[movie_indices]['title']
    
    return pd.DataFrame({'Recommended Movie':movies, 'Similarity Score':movie_scores})

In [18]:
get_recommendation('Minions')

,Recommended Movie,Similarity Score
4939,Keanu,0.199843
7630,Banana,0.117551
10774,Stuart Little 3: Call of the Wild,0.111974
1253,Teenage Mutant Ninja Turtles: Out of the Shadows,0.110506
14200,Sherlock Holmes and the Secret Weapon,0.108087
4680,Hideaway,0.107185
12302,Binky Nelson Unpacified,0.106481
12906,Postman Pat: The Movie,0.103343
10517,He Was a Quiet Man,0.101460
10976,10 Items or Less,0.098410


In [19]:
get_recommendation('Iron Man')

,Recommended Movie,Similarity Score
1517,Willy Wonka & the Chocolate Factory,0.205588
48,Schindler's List,0.199891
2833,Romantics Anonymous,0.165724
8236,Epic Movie,0.157284
53,Spirited Away,0.146000
3939,Chocolat,0.121927
7883,Everyone's Hero,0.111751
1404,24 Hour Party People,0.111261
4927,The Kid with a Bike,0.109171
7814,True Memoirs of an International Assassin,0.105875


## 2. Based on Credits, Genres and Keywords

In [20]:
# merge df3, df4 and df5
data = pd.merge(df3, df4, on='id')
data = pd.merge(data, df5, on='id')
data = data.sort_values('vote_count', ascending=False).drop_duplicates(subset='id').reset_index()
data[['title', 'genres', 'cast', 'crew', 'keywords']].head()

,title,genres,cast,crew,keywords
0,Inception,Action Thriller Science Fiction Mystery Adventure,LeonardoDiCaprio JosephGordon-Levitt EllenPage...,HansZimmer ChristopherNolan ChristopherNolan C...,loss of lover dream kidnapping sleep subconsci...
1,The Dark Knight,Drama Action Crime Thriller,ChristianBale MichaelCaine HeathLedger AaronEc...,LucindaSyson HansZimmer CharlesRoven Christoph...,dc comics crime fighter secret identity scarec...
2,Avatar,Action Adventure Fantasy Science Fiction,SamWorthington ZoeSaldana SigourneyWeaver Step...,StephenE.Rivkin RickCarter ChristopherBoyes Ch...,culture clash future space war space colony so...
3,The Avengers,Science Fiction Action Adventure,RobertDowneyJr. ChrisEvans MarkRuffalo ChrisHe...,AlanSilvestri ChristopherBoyes ChristopherBoye...,new york shield marvel comic superhero based o...
4,Deadpool,Action Adventure Comedy,RyanReynolds MorenaBaccarin EdSkrein T.J.Mille...,RobertAlonzo TimMiller RhettReese PaulWernick ...,anti hero mercenary marvel comic superhero bas...


In [21]:
data['text'] = data['genres']+" "+data['cast']+" "+data['crew']+" "+data['keywords']

In [22]:
data = data.iloc[:20000].copy()

# drop duplicates based on title
data.drop_duplicates(subset=['title'], inplace=True)

# replace NaN values with empty string
data['text'] = data['text'].fillna('')

# cleaning the text
data['text'] = data['text'].progress_apply(text_process)

100%|██████████| 19071/19071 [02:41<00:00, 118.24it/s]


In [23]:
# convert overview column data to matrix to be able to compute similarity after removing the stop words
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=0.0001)
tfidf_matrix = tfidf.fit_transform(data['text'])

print(tfidf_matrix.shape)

(19071, 133581)


In [24]:
# compute similarity matrix
# options available - euclidean distance, the Pearson Coeffiecient, cosine similarity score

# compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# create reverse title indices series
indices = pd.Series(data.index, index=data['title'])

In [25]:
get_recommendation("Tangled")

,Recommended Movie,Similarity Score
583,Bolt,0.319383
2515,Tangled Ever After,0.317820
868,The Princess and the Frog,0.152309
734,Enchanted,0.126866
6060,A Cinderella Story: If the Shoe Fits,0.118172
847,Sleeping Beauty,0.111053
1155,Chicken Little,0.108881
11150,Prep & Landing,0.101985
220,Aladdin,0.100159
2387,Another Cinderella Story,0.097008


In [26]:
get_recommendation('Spectre')

,Recommended Movie,Similarity Score
24,Skyfall,0.252677
291,Quantum of Solace,0.181519
1003,Die Another Day,0.159738
1276,The World Is Not Enough,0.103503
184,Casino Royale,0.096311
13682,Everything or Nothing,0.087342
16751,Stoned,0.078104
1282,Johnny English,0.073981
247,Mission: Impossible - Rogue Nation,0.069224
1946,Octopussy,0.066787


# Recommender 3: Collaborative Filtering
Based on records from various users provide recommendations based on user similarities

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

## 1. Using mean of other user' weighted ratings based on similarity matrix

In [28]:
# merge df1 and df5 to get movie titles and drop rows for which title is not available
data = pd.merge(df1, df5[['id', 'title']], left_on='movieId', right_on='id')

# get total counts of no. of occurence of movie
data['count'] = data.groupby('movieId').transform('count')['userId']

# fetch top 100 movies based on count
movieId = data.drop_duplicates('movieId').sort_values(
    'count', ascending=False).iloc[:100]['movieId']

# filter out data as per the movieId
data = data[data['movieId'].isin(movieId)].reset_index(drop=True)

# get total counts of movies each user has seen
data['count'] = data.groupby('userId').transform('count')['movieId']

# fetch top 20000 users based on no. of movies watched
userId = data.drop_duplicates('userId').sort_values(
    'count', ascending=False).iloc[:20001]['userId']

# filter out data as per the userId
data = data[data['userId'].isin(userId)].reset_index(drop=True)

In [29]:
# create a user movie rating matrix
df = data.pivot(index='userId', columns='movieId', values='rating')
df.head()

movieId,2,6,17,19,21,25,62,95,110,111,...,4011,4226,4886,4896,4973,4993,4995,8961,44191,58559
userId,,,,,,,,,,,,,,,,,,,,,
24,3.0,4.0,3.0,NaN,4.0,3.0,2.0,NaN,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.5,...,5.0,5.0,1.5,NaN,4.0,4.5,3.0,3.5,2.0,4.0
46,NaN,3.0,NaN,3.5,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,4.5,NaN,NaN,4.0,NaN,4.5,NaN,NaN
49,4.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,4.0,5.0,...,NaN,5.0,3.0,5.0,4.0,5.0,NaN,4.0,5.0,NaN
62,2.0,NaN,NaN,4.0,NaN,NaN,3.5,NaN,NaN,4.0,...,4.5,4.0,3.5,4.0,NaN,4.0,NaN,NaN,NaN,NaN


In [30]:
# replace NaN with user based average rating
df_imputed = df.fillna(df.mean(axis=0))

# get similarity between all users
similarity_matrix = cosine_similarity(df_imputed.values)

In [31]:
def get_recommendation(user_index):
    idx = user_index
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # get movies that are unrated by the given user
    unrated_movies = df.iloc[idx][df.iloc[idx].isna()].index

    # get weighted ratings of unrated movies by all other users
    movie_ratings = (df_imputed.iloc[similarity_matrix[idx]][unrated_movies].T * [
        x[1] for x in sim_scores]).T

    # get top 100 similar users by skipping the current user
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:101]

    # get mean of movie rating by top 100 most similar users for the unrated movies
    movie_ratings = movie_ratings.iloc[[x[0] for x in sim_scores]].mean()

    # get recommended movie titles in sorted order
    recommended_movies = df5[df5['id'].isin(movie_ratings.reset_index().sort_values(
        0, ascending=False)['movieId'])][['title', 'id']]
    assumed_ratings = sorted(movie_ratings, reverse=True)

    return pd.DataFrame({'movieId':recommended_movies[:10]['id'], 
                         'Recommended Movie':recommended_movies[:10]['title'], 
                         'Assumed Rating':assumed_ratings[:10]})

In [32]:
user_id = 101
recommended_movies = get_recommendation(user_id).reset_index(drop=True)
# get other high rated movies by user
temp = data[data['userId']==df.index[user_id]].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
recommended_movies['userId'] = temp['userId']
recommended_movies['Movie Watched'] = temp['title']
recommended_movies['Rated']= temp['rating']
recommended_movies

,movieId,Recommended Movie,Assumed Rating,userId,Movie Watched,Rated
0,6,Judgment Night,4.932211,1156,The Man with the Golden Arm,5.0
1,329,Jurassic Park,4.932211,1156,Terminator 3: Rise of the Machines,5.0
2,62,2001: A Space Odyssey,4.932211,1156,5 Card Stud,5.0
3,223,Rebecca,4.932211,1156,Sleepless in Seattle,5.0
4,454,Romeo + Juliet,4.932211,1156,48 Hrs.,5.0
5,500,Reservoir Dogs,4.932211,1156,Rope,5.0
6,185,A Clockwork Orange,4.932211,1156,Beauty and the Beast,5.0
7,595,To Kill a Mockingbird,4.085496,1156,The Million Dollar Hotel,5.0
8,539,Psycho,3.945768,1156,The 39 Steps,5.0
9,288,High Noon,3.945768,1156,Solaris,5.0


In [33]:
user_id = 15554
recommended_movies = get_recommendation(user_id).reset_index(drop=True)
# get other high rated movies by user
temp = data[data['userId']==df.index[user_id]].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
recommended_movies['userId'] = temp['userId']
recommended_movies['Movie Watched'] = temp['title']
recommended_movies['Rated']= temp['rating']
recommended_movies

,movieId,Recommended Movie,Assumed Rating,userId,Movie Watched,Rated
0,6,Judgment Night,4.947474,210430,License to Wed,5.0
1,329,Jurassic Park,4.947474,210430,The Prisoner of Zenda,5.0
2,62,2001: A Space Odyssey,4.947474,210430,High Noon,4.5
3,454,Romeo + Juliet,4.947474,210430,Monsieur Hulot's Holiday,4.5
4,500,Reservoir Dogs,4.947474,210430,The Man with the Golden Arm,4.5
5,185,A Clockwork Orange,4.132242,210430,Say Anything...,4.5
6,595,To Kill a Mockingbird,4.103356,210430,Once Were Warriors,4.5
7,539,Psycho,3.957979,210430,The Bourne Supremacy,4.5
8,339,Night on Earth,3.957979,210430,Beetlejuice,4.5
9,1968,Fools Rush In,3.957979,210430,The Talented Mr. Ripley,4.0


## 2. Using Surprise library

In [34]:
from surprise import Reader, Dataset, SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, \
KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import train_test_split, cross_validate

In [35]:
reader = Reader(rating_scale=(0, 10))
surprise_data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(surprise_data, test_size=0.25)

In [36]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD()]:
#KNNBaseline(), CoClustering(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), 
#BaselineOnly()
    # Perform cross validation
    results = cross_validate(algorithm, surprise_data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp) 
    print(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

test_rmse     0.76989
fit_time     85.92201
test_time    5.669261
Algorithm         SVD
dtype: object


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.76989,85.92201,5.669261


In [37]:
svd = SVD() 
svd.fit(trainset)

In [38]:
index_val = 101
userId = df.index[index_val]
movies = []
ratings = []
titles = []

for movieId in df.iloc[index_val][df.iloc[index_val].isna()].index:
    movies.append(movieId)
    title = df5[df5['id']==movieId]['title'].values[0]
    titles.append(title)
    ratings.append(svd.predict(userId, movieId).est)

prediction = pd.DataFrame({'movieId':movies, 'title':titles, 'rating':ratings, 'userId':userId}) 
prediction = prediction.sort_values('rating', ascending=False).reset_index(drop=True).iloc[:10]

# get other high rated movies by user
temp = data[data['userId']==userId].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
prediction['Movie Watched'] = temp['title']
prediction['Rated']= temp['rating']
prediction

,movieId,title,rating,userId,Movie Watched,Rated
0,17,The Dark,4.123444,1156,The Man with the Golden Arm,5.0
1,919,Blood: The Last Vampire,4.113911,1156,Terminator 3: Rise of the Machines,5.0
2,1213,The Talented Mr. Ripley,4.057264,1156,5 Card Stud,5.0
3,595,To Kill a Mockingbird,4.044896,1156,Sleepless in Seattle,5.0
4,1584,School of Rock,4.037627,1156,48 Hrs.,5.0
5,364,Batman Returns,4.024408,1156,Rope,5.0
6,1246,Rocky Balboa,4.023949,1156,Beauty and the Beast,5.0
7,6,Judgment Night,4.012455,1156,The Million Dollar Hotel,5.0
8,4973,Under the Sand,4.011581,1156,The 39 Steps,5.0
9,1961,My Name Is Bruce,4.003065,1156,Solaris,5.0


In [39]:
index_val = 15554
userId = df.index[index_val]
df.iloc[index_val][df.iloc[index_val].isna()].index
movies = []
ratings = []
titles = []

for movieId in df.iloc[index_val][df.iloc[index_val].isna()].index:
    movies.append(movieId)
    title = df5[df5['id']==movieId]['title'].values[0]
    titles.append(title)
    ratings.append(svd.predict(userId, movieId).est)

prediction = pd.DataFrame({'movieId':movies, 'title':titles, 'rating':ratings, 'userId':userId})  
prediction = prediction.sort_values('rating', ascending=False).reset_index(drop=True).iloc[:10]

# get other high rated movies by user
temp = data[data['userId']==userId].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
prediction['Movie Watched'] = temp['title']
prediction['Rated']= temp['rating']
prediction

,movieId,title,rating,userId,Movie Watched,Rated
0,1089,Point Break,4.252339,210430,License to Wed,5.0
1,2324,Local Color,4.203694,210430,The Prisoner of Zenda,5.0
2,58559,Confession of a Child of the Century,4.160858,210430,High Noon,4.5
3,912,The Thomas Crown Affair,4.103079,210430,Monsieur Hulot's Holiday,4.5
4,551,The Poseidon Adventure,4.092581,210430,The Man with the Golden Arm,4.5
5,111,Scarface,4.082878,210430,Say Anything...,4.5
6,1968,Fools Rush In,3.986412,210430,Once Were Warriors,4.5
7,924,Dawn of the Dead,3.981295,210430,The Bourne Supremacy,4.5
8,1246,Rocky Balboa,3.973850,210430,Beetlejuice,4.5
9,1259,Notes on a Scandal,3.909606,210430,The Talented Mr. Ripley,4.0


# 3. Using LightFM
Using the user ratings along with the movie overview

In [40]:
from scipy.sparse import csr_matrix
from lightfm import LightFM

In [41]:
# prepare item features
moviesId = df.columns

# get overview of the movies
movies_metadata = df5[df5['id'].isin(moviesId)]

# replace NaN values with empty string
movies_metadata['overview'] = movies_metadata['overview'].fillna('')

# cleaning the text
movies_metadata['overview'] = movies_metadata['overview'].progress_apply(text_process)

# convert overview column data to matrix
tfidf = TfidfVectorizer()#ngram_range=(1,1), min_df=0.0001)
tfidf_matrix = tfidf.fit_transform(movies_metadata['overview'])

print(tfidf_matrix.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 100/100 [00:00<00:00, 130.22it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(100, 1873)


In [42]:
csr_data = csr_matrix(df_imputed.values) 
csr_data

<20001x100 sparse matrix of type '<class 'numpy.float64'>'
	with 2000100 stored elements in Compressed Sparse Row format>

In [43]:
# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(csr_data, item_features=tfidf_matrix, epochs=15, num_threads=2, verbose=True)

Epoch: 100%|██████████| 15/15 [01:27<00:00,  5.84s/it]


In [44]:
def get_recommendation(user_index):
    idx = user_index
    # get scores for all the movies
    n_users, n_items = csr_data.shape
    scores = pd.Series(model.predict(0, np.arange(n_items), item_features=tfidf_matrix))
    scores.index = df.columns
    scores = scores.reset_index()

    # get movies that are unrated by the given user
    unrated_movies = df.iloc[idx][df.iloc[idx].isna()].index

    scores = scores[scores['movieId'].isin(unrated_movies)].reset_index(drop=True)
    scores = scores.sort_values(0, ascending=False).iloc[:10]

    # get recommended movie titles in sorted order
    recommended_movies = df5[df5['id'].isin(scores.movieId)][['title', 'id']]

    return pd.DataFrame({'movieId':recommended_movies['id'], 
                         'Recommended Movie':recommended_movies['title']})

In [45]:
user_id = 1156
temp = df.reset_index()
user_idx = int(temp[temp['userId']==user_id].index[0])
print('index val:', user_idx)
recommended_movies = get_recommendation(user_idx).reset_index(drop=True)
# get other high rated movies by user
temp = data[data['userId']==user_id].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
recommended_movies['userId'] = temp['userId']
recommended_movies['Movie Watched'] = temp['title']
recommended_movies['Rated']= temp['rating']
recommended_movies

index val: 101


,movieId,Recommended Movie,userId,Movie Watched,Rated
0,6,Judgment Night,1156,The Man with the Golden Arm,5.0
1,4011,Beetlejuice,1156,Terminator 3: Rise of the Machines,5.0
2,1923,Twin Peaks: Fire Walk with Me,1156,5 Card Stud,5.0
3,608,Men in Black II,1156,Sleepless in Seattle,5.0
4,153,Lost in Translation,1156,48 Hrs.,5.0
5,316,Grill Point,1156,Rope,5.0
6,25,Jarhead,1156,Beauty and the Beast,5.0
7,231,Syriana,1156,The Million Dollar Hotel,5.0
8,2355,Reign Over Me,1156,The 39 Steps,5.0
9,2012,Silent Light,1156,Solaris,5.0


In [46]:
user_id = 210430
temp = df.reset_index()
user_idx = int(temp[temp['userId']==user_id].index[0])
print('index val:', user_idx)
recommended_movies = get_recommendation(user_idx).reset_index(drop=True)
# get other high rated movies by user
temp = data[data['userId']==user_id].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
recommended_movies['userId'] = temp['userId']
recommended_movies['Movie Watched'] = temp['title']
recommended_movies['Rated']= temp['rating']
recommended_movies

index val: 15554


,movieId,Recommended Movie,userId,Movie Watched,Rated
0,6,Judgment Night,210430,License to Wed,5.0
1,1089,Point Break,210430,The Prisoner of Zenda,5.0
2,161,Ocean's Eleven,210430,High Noon,4.5
3,153,Lost in Translation,210430,Monsieur Hulot's Holiday,4.5
4,316,Grill Point,210430,The Man with the Golden Arm,4.5
5,25,Jarhead,210430,Say Anything...,4.5
6,231,Syriana,210430,Once Were Warriors,4.5
7,2355,Reign Over Me,210430,The Bourne Supremacy,4.5
8,2012,Silent Light,210430,Beetlejuice,4.5
9,1387,Superstar: The Karen Carpenter Story,210430,The Talented Mr. Ripley,4.0
